In [ ]:
from os import getenv
from pathlib import Path

input_folder = Path(getenv(
    'CROSSCOMPUTE_INPUT_FOLDER', 'tests/standard/input'))
output_folder = Path(getenv(
    'CROSSCOMPUTE_OUTPUT_FOLDER', 'tests/standard/output'))

output_folder.mkdir(parents=True, exist_ok=True)

In [ ]:
import json

with (input_folder / 'variables.dictionary').open('rt') as f:
    d = json.load(f)
districts_uri = d['districts_uri']

Use a hashing function to get a unique and safe string based on user input.

In [ ]:
from hashlib import blake2b

def get_hash(text):
    h = blake2b()
    h.update(text.encode())
    return h.hexdigest()

In [ ]:
from shapely.geometry import mapping, shape

def simplify_feature(feature, tolerance):
    raw_geometry = shape(feature['geometry'])
    simplified_geometry = raw_geometry.simplify(tolerance)
    feature['geometry'] = mapping(simplified_geometry)
    return feature

## Exercise 1-1

Set simplification tolerance to 0.1 and observe how the map changes.

In [ ]:
import json
from urllib.request import urlretrieve as download_uri

SIMPLIFICATION_TOLERANCE = 0.001

datasets_folder = Path('datasets')
districts_path = (
    datasets_folder / 'districts' / get_hash(districts_uri)
).with_suffix('.json')
if not districts_path.exists():
    districts_path.parent.mkdir(parents=True, exist_ok=True)
    raw_path = districts_path.with_suffix('.raw')
    download_uri(districts_uri, raw_path)
    with raw_path.open('rt') as f:
        d = json.load(f)
        d['features'] = features = [simplify_feature(
            _, SIMPLIFICATION_TOLERANCE) for _ in d['features']]
    with districts_path.open('wt') as f:
        json.dump(d, f)
else:
    with districts_path.open('rt') as f:
        d = json.load(f)
        features = d['features']
districts_geojson = d
len(features)

In [ ]:
# ls $districts_path.parent -l -h

In [ ]:
# rm $districts_path

# Exercise 1-2
Change the color map from blue/red to green/yellow in `automate.yml`.

# Exercise 1-3

Define a new variable called `x` that is random from 0 to 100.

# Exercise 1-4
Change the color map to vary from 0 to 100  in `automate.yml`.

In [ ]:
from random import choice

for feature in features:
    feature['properties'] = {'t': choice(range(1, 60))}

In [ ]:
with (output_folder / 'map.geojson').open('wt') as f:
    json.dump(districts_geojson, f)

# Exercise 1-5

[Replace community districts with fire divisions](https://www1.nyc.gov/site/planning/data-maps/open-data/districts-download-metadata.page).